<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/38_3_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Установка необходимых пакетов
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install protobuf~=3.20.3
!pip install -q tensorflow_datasets
!pip install -q -U tensorflow-text tensorflow

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package libcudnn8 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Version '8.1.0.77-1+cuda11.2' for 'libcudnn8' was not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.1 MB/s eta 0:00:00


In [3]:
# Импорт необходимых библиотек
import logging
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text
import re
import pathlib
import random
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Загрузка датасета
ds = tfds.load('web_questions', split='train', shuffle_files=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3778 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.VYOLTR_1.0.0/web_questions-train.tfrecord*...:   …

Generating test examples...:   0%|          | 0/2032 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/web_questions/incomplete.VYOLTR_1.0.0/web_questions-test.tfrecord*...:   0…

Dataset web_questions downloaded and prepared to /root/tensorflow_datasets/web_questions/1.0.0. Subsequent calls will reuse this data.


In [5]:
# Содержимое датасета
ds

<_PrefetchDataset element_spec={'answers': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'question': TensorSpec(shape=(), dtype=tf.string, name=None), 'url': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [6]:
# Выводим три примера из датасета
i = 1
for example in ds:
    if i < 4:
        print(f"Пример из обучающей выборки №{i}:")
        print(*example['answers'].numpy())
        print(example['question'].numpy())
        print(example['url'].numpy(), "\n")
        i += 1

Пример из обучающей выборки №1:
b'Joe Flacco'
b'who is the ravens quarterback 2012?'
b'http://www.freebase.com/view/en/baltimore_ravens' 

Пример из обучающей выборки №2:
b'Whig Party'
b'what political party was henry clay?'
b'http://www.freebase.com/view/en/henry_clay' 

Пример из обучающей выборки №3:
b'UTC+07:00'
b'where to stay in thailand koh samui?'
b'http://www.freebase.com/view/en/ko_samui' 



In [7]:
# создаем списки вопросов и ответов
question_list = []
answer_list = []

i = 0
for example in ds:
    if i < 6642:
        question = example['question'].numpy()
        question_list.append(question)

        answer = example['answers'].numpy()
        answer_list.append(answer)
        i += 1

print('Размер списка вопросов: ', len(question_list))
print('Размер списка ответов: ', len(answer_list))

Размер списка вопросов:  3778
Размер списка ответов:  3778


In [8]:
train_examples_question = question_list[:3000]
train_examples_answer = answer_list[:3000]

val_examples_question = question_list[3001:]
val_examples_answer = answer_list[3001:]

print(f"Длина обучающей (вопросы): {len(train_examples_question)}")
print(f"Длина обучающей (ответы): {len(train_examples_answer)}")
print(f"Длина валидационной (вопросы): {len(val_examples_question)}")
print(f"Длина обучаювалидационнойщей (ответы): {len(val_examples_answer)}")

Длина обучающей (вопросы): 3000
Длина обучающей (ответы): 3000
Длина валидационной (вопросы): 777
Длина обучаювалидационнойщей (ответы): 777


In [9]:
# Выводим содержимое спика
rnd = random.randint(0, 3000)
print(train_examples_question[rnd])
print(train_examples_answer[rnd])

b'what are the islands of oceania?'
[b'Northern Mariana Islands' b'Norfolk Island' b'Guam' b'American Samoa'
 b'New Caledonia' b'Marshall Islands' b'Kiribati' b'Nauru' b'Fiji'
 b'Cook Islands']


In [10]:
# токенизация списков вопросов и ответов
VOCAB_SIZE = 8000
bert_tokenizer_params=dict(lower_case=True)

reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

bert_vocab_args = dict(
    vocab_size = VOCAB_SIZE,
    reserved_tokens=reserved_tokens,
    bert_tokenizer_params=bert_tokenizer_params,
    learn_params={},)

In [12]:
train_question = train_examples_question.map(lambda pt, en: en)
train_answer = train_examples_answer.map(lambda pt, en: pt)

AttributeError: 'list' object has no attribute 'map'

In [14]:
# ожидает: <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))
# возвращает: <_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

train_question = ds.map(lambda train_examples_question, train_examples_answer: train_examples_answer)
train_answer = ds.map(lambda train_examples_question, train_examples_answer: train_examples_question)

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.<lambda>() missing 1 required positional argument: 'train_examples_answer'
